# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [ ]:
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [ ]:
admission_predict = pd.read_csv(r'C:\Users\Daniel Carvalho\Desktop\DataAnalysis\WEEK2\DAY3\labs\lab-advanced-pandas\your-code\data\Admission_Predict.csv')
admission_predict.head()

Remove trailing spaces at the end of the column names if there are any.


In [ ]:
admission_predict.columns = admission_predict.columns.str.strip()

Let's evaluate the dataset by looking at the `head` function.

In [ ]:
admission_predict.head()

Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [ ]:
admission_predict.notnull() #need to check this


Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [ ]:
pd.DataFrame.set_index(admission_predict,'Serial No.')

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [ ]:
print(admission_predict.set_index(['GRE Score','CGPA']).index.is_unique)

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [ ]:
admission_predict['GRE SCORE_CGPA'] = admission_predict.groupby(['GRE Score','CGPA'])
admission_predict.drop(columns=['GRE Score', 'CGPA'])
pd.DataFrame.set_index(admission_predict,'GRE SCORE_CGPA')

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [ ]:
admission_predict.reset_index()

# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [ ]:
admission_predict[(admission_predict['CGPA'] > 9) & (admission_predict['Research'] == 1)]

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [ ]:
adm = admission_predict[(admission_predict['CGPA'] > 9) & (admission_predict['SOP'] < 3.5)]

adm.mean()


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [ ]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    result = admission_predict['col']-admission_predict['col'].mean()/admission_predict['col'].std()
    return result

In [ ]:
admission_predict

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [85]:
admission_predict['GRE_std'] = (admission_predict['GRE Score']-admission_predict['GRE Score'].mean())/admission_predict['GRE Score'].std()
admission_predict['CGPA_std']=(admission_predict['CGPA']-admission_predict['CGPA'].mean())/admission_predict['CGPA'].std()
admission_predict['LOR_std']=(admission_predict['LOR']-admission_predict['LOR'].mean())/admission_predict['LOR'].std()

We will generate the decision choice at random using the code below. Please run the cell.

In [ ]:
# Libraries
from random import choices

In [82]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']
decision_choice = choices(std_columns, k=admission_predict.shape[0])
admission_predict

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,GRE SCORE_CGPA,CGPA_std,Gre_std,LOR_std,decision_column,adjusted_gre
0,1,337,118,4,4.5,4.5,9.65,1,0.92,"((290, 7.46), [Serial No.])",1.747900,1.753381,1.191647,GRE_std,347
1,2,316,104,3,3.0,3.5,8.00,1,0.72,"((290, 7.56), [Serial No.])",-0.991211,-0.063367,0.076740,GRE_std,326
2,3,322,110,3,3.5,2.5,8.67,1,0.80,"((293, 7.8), [Serial No.])",0.121034,0.455704,-1.038166,GRE_std,332
3,4,314,103,2,2.0,3.0,8.21,0,0.65,"((294, 7.36), [Serial No.])",-0.642597,-0.236391,-0.480713,GRE_std,324
4,5,330,115,5,4.5,3.0,9.34,1,0.90,"((294, 7.64), [Serial No.])",1.233279,1.147798,-0.480713,GRE_std,340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,"((340, 9.6), [Serial No.])",0.735259,0.628727,0.076740,CGPA_std,334
381,382,325,107,3,3.0,3.5,9.11,1,0.84,"((340, 9.66), [Serial No.])",0.851463,0.715239,0.076740,CGPA_std,335
382,383,330,116,4,5.0,4.5,9.45,1,0.91,"((340, 9.74), [Serial No.])",1.415886,1.147798,1.191647,CGPA_std,340
383,384,312,103,3,3.5,4.0,8.78,0,0.67,"((340, 9.91), [Serial No.])",0.303641,-0.409414,0.634193,GRE_std,322


Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [86]:
admission_predict['decision_column'] = admission_predict.lookup(admission_predict.index,decision_choice)

C:\Users\Daniel Carvalho\AppData\Local\Temp\ipykernel_19392\4038816001.py:1: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  admission_predict['decision_column'] = admission_predict.lookup(admission_predict.index,decision_choice)


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [ ]:
admission_predict['deciding_column'] = decision_choice

for i in admission_predict['deciding_column']:
    if i >= 0.8:
        admission_predict['Decision'] == 1
    else:
        admission_predict['Decision'] == 0

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [ ]:
for i in admission_predict['Decision']:
    if i == 1:
        print(i)
    else:
        continue

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [ ]:
admission_predict.columns.str.replace(' ', '_').lower()

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

----------------------------------------------------------- ITS ADDING TO 10 TO ALL ROWS EVEN IF UNI RATING BELOW 4----------------------

In [80]:
for i in admission_predict['University Rating']:
    if i >= 4:
        admission_predict['adjusted_gre'] =  admission_predict['GRE Score'] + 10
    else:
        continue


In [75]:
admission_predict['adjusted_gre']
bins = pd.cut(admission_predict['adjusted_gre'], 4)
bins.value_counts()

(325.0, 337.5]     140
(312.5, 325.0]     120
(337.5, 350.0]      70
(299.95, 312.5]     55
Name: adjusted_gre, dtype: int64